In [1]:
%pip install ../../../

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=67507 sha256=30310e7ea7441d63b6b354b36c667744e726fa897cfbf8973542c55c8a62fab7
  Stored in directory: /tmp/pip-ephem-wheel-cache-44pfbhrz/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()

In [5]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [6]:
from spuco.models import model_factory 
model = model_factory("lenet", trainset[0][0].shape, trainset.num_spurious).to(device)

In [7]:
from spuco.group_inference import SSA
from spuco.utils import SpuriousTargetDataset

ssa = SSA(
    spurious_unlabeled_dataset=trainset,
    spurious_labeled_dataset=SpuriousTargetDataset(valset, valset.spurious),
    model=model,
    labeled_valset_size=0.2,
    num_iters=100,
    tau_g_min=0.1,
    device=device,
    verbose=True
)

In [8]:
group_partition = ssa.infer_groups()
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

Training SSA Model 0:   1%|          | 1/100 [00:00<01:16,  1.30it/s, loss=nan, val_acc=20.2]

tensor([[-0.0862, -0.0708,  0.0450, -0.0856, -0.0986],
        [-0.0902, -0.0763,  0.0366, -0.0857, -0.0990],
        [-0.0872, -0.0801,  0.0437, -0.0815, -0.0926],
        [-0.0885, -0.0773,  0.0451, -0.0861, -0.0979],
        [-0.0893, -0.0774,  0.0502, -0.0885, -0.0991],
        [-0.0831, -0.0735,  0.0382, -0.0961, -0.0943],
        [-0.0903, -0.0746,  0.0412, -0.0871, -0.0969],
        [-0.0883, -0.0773,  0.0385, -0.0838, -0.0979],
        [-0.0900, -0.0729,  0.0431, -0.0798, -0.0971],
        [-0.0886, -0.0771,  0.0456, -0.0826, -0.0969],
        [-0.0866, -0.0718,  0.0382, -0.0875, -0.1005],
        [-0.0937, -0.0854,  0.0536, -0.0795, -0.1034],
        [-0.0916, -0.0786,  0.0484, -0.0866, -0.1017],
        [-0.0855, -0.0728,  0.0444, -0.0875, -0.0925],
        [-0.0909, -0.0809,  0.0494, -0.0824, -0.1027],
        [-0.0885, -0.0738,  0.0413, -0.0915, -0.0942],
        [-0.0898, -0.0707,  0.0459, -0.0822, -0.1002],
        [-0.0923, -0.0808,  0.0482, -0.0876, -0.0983],
        [-

Training SSA Model 0:   7%|▋         | 7/100 [00:01<00:09,  9.88it/s, loss=nan, val_acc=20.2]

tensor([[-0.0883, -0.0818,  0.0473, -0.0833, -0.0985],
        [-0.0886, -0.0731,  0.0333, -0.0838, -0.0967],
        [-0.0882, -0.0750,  0.0376, -0.0849, -0.0989],
        [-0.0813, -0.0749,  0.0413, -0.0867, -0.0922],
        [-0.0876, -0.0744,  0.0460, -0.0896, -0.0981],
        [-0.0902, -0.0837,  0.0476, -0.0838, -0.1024],
        [-0.0882, -0.0718,  0.0449, -0.0910, -0.0944],
        [-0.0958, -0.0804,  0.0465, -0.0890, -0.0993],
        [-0.0861, -0.0721,  0.0412, -0.0864, -0.0963],
        [-0.0875, -0.0743,  0.0394, -0.0879, -0.0925],
        [-0.0972, -0.0810,  0.0486, -0.0868, -0.1012],
        [-0.0934, -0.0830,  0.0471, -0.0854, -0.0989],
        [-0.0893, -0.0777,  0.0371, -0.0883, -0.0989],
        [-0.0885, -0.0763,  0.0442, -0.0838, -0.0977],
        [-0.0904, -0.0808,  0.0496, -0.0833, -0.0967],
        [-0.0915, -0.0819,  0.0434, -0.0829, -0.0997],
        [-0.0899, -0.0750,  0.0425, -0.0865, -0.0968],
        [-0.0865, -0.0845,  0.0464, -0.0842, -0.0967],
        [-

Training SSA Model 0:  13%|█▎        | 13/100 [00:01<00:05, 16.85it/s, loss=nan, val_acc=20.2]

...

Training SSA Model 0:  73%|███████▎  | 73/100 [00:03<00:00, 27.79it/s, loss=nan, val_acc=20.2]

tensor([[-0.1339, -0.1264, -0.0623,  0.0096, -0.0497],
        [-0.1269, -0.0787, -0.0174, -0.0120, -0.1043],
        [-0.1258, -0.0714, -0.0312, -0.0114, -0.1006],
        [-0.1396, -0.1256, -0.0691,  0.0065, -0.0563],
        [-0.1503, -0.1245, -0.0711,  0.0113, -0.0626],
        [-0.0663, -0.0776, -0.0615, -0.0349, -0.0588],
        [-0.0696, -0.0788, -0.0532, -0.0327, -0.0667],
        [-0.1916, -0.1408, -0.0385,  0.0502, -0.0963],
        [-0.1417, -0.1298, -0.0706,  0.0190, -0.0559],
        [-0.1329, -0.1243, -0.0608,  0.0115, -0.0500],
        [-0.1428, -0.1230, -0.0722,  0.0058, -0.0510],
        [-0.1496, -0.1027, -0.0152,  0.0100, -0.1089],
        [-0.1051, -0.0856, -0.0671, -0.0221, -0.0673],
        [-0.1679, -0.1053, -0.0112,  0.0303, -0.1180],
        [-0.1675, -0.1047, -0.0125,  0.0282, -0.1155],
        [-0.1018, -0.0946, -0.0614, -0.0106, -0.0589],
        [-0.1340, -0.0689, -0.0247, -0.0019, -0.1053],
        [-0.1357, -0.0846, -0.0179,  0.0002, -0.1060],
        [-

Training SSA Model 0:  79%|███████▉  | 79/100 [00:03<00:00, 21.68it/s, loss=nan, val_acc=20.2]


tensor([[-0.1505, -0.1342, -0.0878,  0.0255, -0.0448],
        [-0.2175, -0.1571, -0.0493,  0.0813, -0.0964],
        [-0.1924, -0.1087, -0.0129,  0.0529, -0.1282],
        [-0.0941, -0.0915, -0.0906, -0.0112, -0.0499],
        [-0.1604, -0.1396, -0.0853,  0.0274, -0.0496],
        [-0.1317, -0.0651, -0.0290, -0.0008, -0.1112],
        [-0.1940, -0.1116, -0.0134,  0.0463, -0.1241],
        [-0.1537, -0.1366, -0.0846,  0.0366, -0.0500],
        [-0.0955, -0.0926, -0.0816, -0.0074, -0.0524],
        [-0.1607, -0.0838, -0.0238,  0.0186, -0.1150],
        [-0.1517, -0.0800, -0.0260,  0.0139, -0.1151],
        [-0.1552, -0.1360, -0.0814,  0.0300, -0.0459],
        [-0.1678, -0.1460, -0.0791,  0.0362, -0.0589],
        [-0.1510, -0.1355, -0.0836,  0.0312, -0.0495],
        [-0.1567, -0.0890, -0.0236,  0.0085, -0.1121],
        [-0.2210, -0.1575, -0.0492,  0.0787, -0.0957],
        [-0.2173, -0.1594, -0.0444,  0.0738, -0.0963],
        [-0.2222, -0.1652, -0.0456,  0.0791, -0.0973],
        [-

IndexError: too many indices for tensor of dimension 2

In [ ]:
from spuco.evaluate import Evaluator 

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
from torch.optim import SGD
from spuco.invariant_train import GroupBalanceERM, ClassBalanceERM
from spuco.models import model_factory 

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

In [ ]:
evaluator.worst_group_accuracy

In [ ]:
evaluator.average_accuracy

In [ ]:
evaluator.evaluate_spurious_task()